# COSC-4117: Hidden Markov Model

#1. Simple Markov Model
In this example, we'll use a basic Markov chain with a small number of states and demonstrate how the state probabilities evolve over time: the convergence of probabilities in a Markov model as the number of time steps approaches infinity.

A Markov model is a stochastic model used to model randomly changing systems where it is assumed that future states depend only on the current state, not on the sequence of events that preceded it. This **memoryless** property is known as the Markov property.

In [ ]:
import numpy as np

# Define a simple transition matrix for a Markov chain
# This should be a square matrix where each row sums up to 1
transition_matrix = np.array([
    [0.9, 0.1],  # Probabilities of moving from state 0 (Sun) to state 0 and 1 (Rain)
    [0.3, 0.7]   # Probabilities of moving from state 1 to state 0 and 1
])

# Initial state probabilities
initial_state = np.array([0.5, 0.0])  # Starting in state 0 with probability 1

# Function to simulate the Markov chain
def simulate_markov_chain(transition_matrix, initial_state, steps):
    current_state = initial_state
    for _ in range(steps):
        current_state = np.dot(current_state, transition_matrix)
    return current_state

# Simulate the Markov chain for different time steps to observe convergence
time_steps = [1, 2, 5, 10, 20, 50, 100, 500, 1000]
results = {step: simulate_markov_chain(transition_matrix, initial_state, step) for step in time_steps}

results

{1: array([0.45, 0.05]),
 2: array([0.42, 0.08]),
 5: array([0.38472, 0.11528]),
 10: array([0.37575583, 0.12424417]),
 20: array([0.37500457, 0.12499543]),
 50: array([0.375, 0.125]),
 100: array([0.375, 0.125]),
 500: array([0.375, 0.125]),
 1000: array([0.375, 0.125])}

As you can see, as the number of steps increases, the state probabilities converge to approximately 0.75 for State 0 (Sun) and 0.25 for State 1 (Rain). This convergence demonstrates the property of Markov chains where, given enough time, the system reaches a steady state where the probabilities of being in each state remain constant. This steady state is independent of the initial state probabilities

#2. Hidden Markov Model

Here's the code for simulating belief states in a Hidden Markov Model, given a series of observations (Umbrella or No Umbrella).

In this simulation:

The belief states are updated at each step based on the observed data and the HMM parameters (transition and emission matrices).
The update_belief_state function calculates the new belief state for each observation using the forward algorithm.
The initial state probabilities are set to [0.5, 0.5], indicating an equal likelihood of starting in either state.
When you run this code with the example observations, it outputs the probability distribution over the states at each step, reflecting the evolving belief about the system's state given the observations.

In [ ]:
import numpy as np

# Transition matrix for the Markov chain
transition_matrix = np.array([
    [0.7, 0.3],  # Probabilities of moving from state 0 (Rain) to state 0 and 1 (Sun)
    [0.3, 0.7]   # Probabilities of moving from state 1 to state 0 and 1
])

# Emission matrix for the Hidden Markov Model (HMM)
# Each row corresponds to a state and each column to an observation
emission_matrix = np.array([
    [0.9, 0.1],  # Probabilities of emitting 0 (Umbrella) or 1 (No umbrella) from state 0
    [0.2, 0.8]   # Probabilities of emitting 0 or 1 from state 1
])

# Initial state probabilities
initial_state = np.array([0.5, 0.5])  # Equally likely to start in either state

def update_belief_state(belief_state, observation, transition_matrix, emission_matrix):
    """
    Update the belief state based on an observation, using the forward algorithm.
    """
    num_states = len(belief_state)
    new_belief_state = np.zeros(num_states)

    for state in range(num_states):
        # Sum over all possible previous states
        total = sum(belief_state[prev_state] * transition_matrix[prev_state, state]
                    for prev_state in range(num_states))
        new_belief_state[state] = total * emission_matrix[state, observation]

    # Normalize the new belief state
    new_belief_state /= np.sum(new_belief_state)
    return new_belief_state

def simulate_belief_states(transition_matrix, emission_matrix, initial_state, observations):
    """
    Simulate the belief states for a series of observations.
    """
    belief_states = [initial_state]
    current_belief_state = initial_state

    for observation in observations:
        current_belief_state = update_belief_state(current_belief_state, observation,
                                                   transition_matrix, emission_matrix)
        belief_states.append(current_belief_state)

    return belief_states

# Example observations 0 (Umbrella) or 1 (No umbrella)
example_observations = [0, 0, 0, 1, 1, 1, 1, 1, 1, 0]

# Simulate the belief states for the given observations
belief_states = simulate_belief_states(transition_matrix, emission_matrix, initial_state, example_observations)

# Output the belief states
for step, belief in enumerate(belief_states):
    print(f"Step {step}: State 0 probability = {belief[0]:.4f}, State 1 probability = {belief[1]:.4f}")

Step 0: State 0 probability = 0.5000, State 1 probability = 0.5000
Step 1: State 0 probability = 0.8182, State 1 probability = 0.1818
Step 2: State 0 probability = 0.8834, State 1 probability = 0.1166
Step 3: State 0 probability = 0.8945, State 1 probability = 0.1055
Step 4: State 0 probability = 0.1937, State 1 probability = 0.8063
Step 5: State 0 probability = 0.0705, State 1 probability = 0.9295
Step 6: State 0 probability = 0.0575, State 1 probability = 0.9425
Step 7: State 0 probability = 0.0563, State 1 probability = 0.9437
Step 8: State 0 probability = 0.0562, State 1 probability = 0.9438
Step 9: State 0 probability = 0.0562, State 1 probability = 0.9438
Step 10: State 0 probability = 0.6817, State 1 probability = 0.3183
